<h1 style="color:darkgreen">Análise dos sorteios do NF Premiada MS </h1>

<h2 style="color:darkgray">Oque é o NF premiada MS?</h2>
<h3> É um programa de incentivo de participação fiscal popular em que o governo do estado do Mato Grosso do Sul gera 8 dezenas de 01 a 60 quando o cliente pede o CPF na nota</h3>

<h2 style="color:darkgray"> Como é o sorteio? </h2>
<h3> O sorteio é realizado uma vez por mês, ao final do mês utilizando os números da Mega-sena</h3>

<h2 style="color:darkgray"> E a premiação? </h2>
<h3> A premiação é dividida em: R$ 100.000,00 para os ganhadores das 6 dezenas e R$ 200.000,00 para os ganhadores
das 5 dezenas. Ambos divididos em partes iguais.</h3>

<h2 style="color:darkgray"> Do resgate do prêmio </h2>
<h3> Deve ser realizado um cadastro no site do programa em até 90 dias contados a partir do 15º dia do mês sub-sequente à data do sorteio</h3>
mais informações podem ser vistas no <a href="https://www.notamspremiada.ms.gov.br/">site do programa Nota MS Premiada</a></h3>


## Importação da Biblioteca pandas para trabalhar os dados

In [1]:
import pandas as pd

## Definição de uma função para contar o número de acertos que o premiado teve

In [2]:
def cont_acertos(premiados_dezenas, sorteio_dezenas):
  return len(premiados_dezenas.intersection(sorteio_dezenas))

## Carregar os dados da base listapremiados.xlsx

In [3]:
df_premiados = pd.read_excel('listapremiados.xlsx', parse_dates=['premiados_mes'])

## As dezenas vêm como string todas na mesma célula separadas por espaço, aqui transformamos esta stringem um array

In [4]:
df_premiados['premiados_dezenas'] = df_premiados['premiados_dezenas'].str.split(' ')

## Verificando o tipo dos dados 

In [5]:
df_premiados.dtypes

premiados_mes                datetime64[ns]
premiados_cpf                        object
premiados_dezenas                    object
premiados_estabelecimento            object
premiados_municipio                  object
dtype: object

Carregamento dos dados dezenasSorteadas.xlsx

In [6]:
df_sorteios = pd.read_excel('dezenasSorteadas.xlsx', parse_dates=['sorteio_mes'])

## Verificando os tipos dos dados. O prinicpal é que sorteio_mes seja do tipo datetime pois será a chave de ligação
## entre df_premiados e df_sorteios

In [7]:
df_sorteios.dtypes

sorteio_mes        datetime64[ns]
sorteio_dezenas            object
dtype: object

## Assim como na lista de premiados aqui as dezenas também estão em uma string e sofre o mesmo tratamento

In [8]:
df_sorteios.sorteio_dezenas = df_sorteios.sorteio_dezenas.str.split(' ')

## Criando uma coluna mes em cada dataframe apenas para ser chave de ligação entre elas

In [9]:
df_sorteios['mes'] = df_sorteios['sorteio_mes']
df_premiados['mes'] = df_premiados['premiados_mes']


## Fazendo merge dos dataframes para poder ver os numeros que cada premiado acertou

In [10]:
df_premiados = pd.merge(df_premiados,df_sorteios,on='mes')

## Transofrmando cada valor de _dezenas em um set permitindo usar as funções de conjunto para facilitar a conferencia do sorteio

In [11]:
df_premiados['premiados_dezenas'] = df_premiados['premiados_dezenas'].apply(lambda x: set(x))
df_premiados['sorteio_dezenas'] = df_premiados['sorteio_dezenas'].apply(lambda x: set(x))

## Aplicando a função anteriormente definida -- cont_acertos -- para cada linha e criando a cooluna 'acertos' para armazenar o resultado

In [12]:
df_premiados['acertos'] = df_premiados.apply(lambda x: cont_acertos(x.premiados_dezenas, x.sorteio_dezenas), axis=1)

## Premiados por mês e cidadade
### Agrupando os valores por mês e cidade com a função agregadora 'count' sobre a coluna premiados_cpf
### Rearranjando o dataframe por meio da nlargest para que cada agrupamento respeite a ordem do valor em count

In [13]:
df_premiados_mes_municipio = df_premiados.groupby(['mes','premiados_municipio']).agg({'count'})['premiados_cpf']
df_premiados_mes_municipio = df_premiados_mes_municipio['count'].groupby('mes',group_keys=False).nlargest(999)
df_premiados_mes_municipio

mes         premiados_municipio  
2020-07-01  Campo Grande             142
            Dourados                  39
            Corumbá                   23
            Três Lagoas               20
            Nova Andradina            10
                                    ... 
2021-08-01  Novo Horizonte do Sul      1
            Bonito                     1
            Nova Alvorada do Sul       1
            Caracol                    1
            Itaporã                    1
Name: count, Length: 657, dtype: int64

## Criando uma pivot table a partir de df_premiados_mes_municipio

In [14]:
df_premiados_mes_municipio_pivot = df_premiados
df_premiados_mes_municipio_pivot.fillna(0)
df_premiados_mes_municipio_pivot = pd.pivot_table(
  df_premiados_mes_municipio_pivot,
  index= ['premiados_municipio'],
  columns = ['mes'],
  margins=True,
  values='premiados_cpf',
  aggfunc='count',
  fill_value=0
)
df_premiados_mes_municipio_pivot.sort_values(by=('All'), ascending=False, inplace=True)
df_premiados_mes_municipio_pivot

mes,2020-07-01 00:00:00,2020-08-01 00:00:00,2020-09-01 00:00:00,2020-10-01 00:00:00,2020-11-01 00:00:00,2020-12-01 00:00:00,2021-01-01 00:00:00,2021-02-01 00:00:00,2021-03-01 00:00:00,2021-04-01 00:00:00,2021-05-01 00:00:00,2021-06-01 00:00:00,2021-07-01 00:00:00,2021-08-01 00:00:00,All
premiados_municipio,,,,,,,,,,,,,,,
All,343,365,363,373,394,374,382,345,349,318,350,359,337,383,5035
Campo Grande,142,160,153,170,195,170,170,164,174,132,166,168,149,167,2280
Dourados,39,31,45,46,53,49,42,45,30,35,41,36,31,46,569
Três Lagoas,20,24,19,21,21,16,28,15,17,24,15,21,21,21,283
Corumbá,23,9,14,9,17,16,14,9,13,11,12,18,14,8,187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Figueirão,1,0,0,0,0,0,0,0,0,0,0,1,0,0,2
Aral Moreira,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
Laguna Carapã,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1


## Criando uma pivot table para mostrar por mês e municpio a quantidade de ganhadores que acertaram as 6 dezenas

In [15]:
df_premiados.sort_values(['acertos'], ascending=False).where(df_premiados['acertos'] == 6)
df_ganhadores_6_acertos_mes_municipio = pd.pivot_table(
  df_premiados.sort_values(['acertos'], ascending=False).where(df_premiados['acertos'] == 6),
  index= ['premiados_municipio'],
  columns = ['mes'],
  margins=True,
  values='premiados_cpf',
  aggfunc='count',
  fill_value=0
)
df_ganhadores_6_acertos_mes_municipio

mes,2020-07-01 00:00:00,2020-08-01 00:00:00,2020-09-01 00:00:00,2020-10-01 00:00:00,2020-11-01 00:00:00,2020-12-01 00:00:00,2021-01-01 00:00:00,2021-02-01 00:00:00,2021-03-01 00:00:00,2021-04-01 00:00:00,2021-05-01 00:00:00,2021-06-01 00:00:00,2021-07-01 00:00:00,2021-08-01 00:00:00,All
premiados_municipio,,,,,,,,,,,,,,,
Bela Vista,0,0,0,0,0,1,0,0,1,0,1,0,0,0,3
Caarapó,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
Campo Grande,1,1,1,0,0,4,1,1,1,2,0,1,1,0,14
Corumbá,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
Dourados,0,0,1,0,0,2,0,0,0,0,0,1,0,0,4
Maracaju,0,0,0,1,1,1,0,0,0,0,0,0,0,0,3
Mundo Novo,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
Naviraí,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
Nova Andradina,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1


## Listando por mes e municipio os ganhadores que acertaram as 6 dezenas

In [16]:
df_premiados_6_acertos_cidade = df_premiados.where(df_premiados['acertos'] ==6)
df_premiados_6_acertos_cidade.dropna()
df_premiados_6_acertos_cidade = df_premiados_6_acertos_cidade.groupby(['mes','premiados_municipio']).agg({'count'})['premiados_cpf']
df_premiados_6_acertos_cidade = df_premiados_6_acertos_cidade['count'].groupby('mes',group_keys=False).nlargest(999)
df_premiados_6_acertos_cidade

mes         premiados_municipio 
2020-07-01  Campo Grande            1
2020-08-01  Campo Grande            1
2020-09-01  Campo Grande            1
            Dourados                1
2020-10-01  Maracaju                1
2020-11-01  Corumbá                 1
            Maracaju                1
            Três Lagoas             1
2020-12-01  Campo Grande            4
            Dourados                2
            Bela Vista              1
            Maracaju                1
2021-01-01  Campo Grande            1
            Mundo Novo              1
            Naviraí                 1
            Sidrolândia             1
            Três Lagoas             1
2021-02-01  Campo Grande            1
            Três Lagoas             1
2021-03-01  Bela Vista              1
            Campo Grande            1
2021-04-01  Campo Grande            2
2021-05-01  Bela Vista              1
            São Gabriel do Oeste    1
2021-06-01  Campo Grande            1
            Doura

## Listando por CPF e municipio aqueles que foram premiados mias de uma
### Como o número completo do CPF é sigiloso está sendo admitido que apenas o mesmo código de CPF em combinação com a mesma cidade é uma pessoa única
### Ignorando a possibilidade de o mesmo CPF ganhar em outra cidade, criando asism um novo resigtro

In [17]:
df_sortudos = df_premiados.groupby(['premiados_municipio','premiados_cpf']).agg({'count'})['premiados_dezenas']
df_sortudos = df_sortudos.where(df_sortudos['count'] >1 ).dropna()
df_sortudos

count
premiados_municipio premiados_cpf       
Campo Grande        ***029.931**     2.0
                    ***056.741**     2.0
                    ***077.101**     2.0
                    ***078.571**     2.0
                    ***089.161**     2.0
                    ***105.921**     2.0
                    ***164.701**     2.0
                    ***196.201**     2.0
                    ***243.551**     2.0
                    ***353.071**     2.0
                    ***417.841**     2.0
                    ***419.101**     2.0
                    ***443.371**     2.0
                    ***474.991**     2.0
                    ***499.038**     2.0
                    ***510.501**     2.0
                    ***529.020**     2.0
                    ***603.141**     2.0
                    ***669.108**     2.0
                    ***678.531**     2.0
                    ***710.081**     2.0
                    ***874.187**     2.0
                    ***898.211**     2.0
                    ***915.031**     2.0
                    ***930.011**     2.0
Corumbá             ***691.951**     2.0
Costa Rica          ***982.621**     2.0
Coxim               ***297.771**     2.0
Dourados            ***508.051**     2.0
Mundo Novo          ***263.588**     2.0
                    ***950.081**     2.0
Nova Andradina      ***210.691**     2.0
                    ***762.550**     4.0
Sidrolândia         ***088.501**     3.0
Três Lagoas         ***523.201**     2.0

In [19]:
df_premiados[df_premiados['premiados_cpf'] == '***762.550**']

,premiados_mes,premiados_cpf,premiados_dezenas,premiados_estabelecimento,premiados_municipio,mes,sorteio_mes,sorteio_dezenas,acertos
1028,2021-06-01,***762.550**,"{16, 39, 15, 13, 50, 49, 35, 08}",AUTO POSTO MAIS COMERCIO DE COMBUSTIVEIS E LUB...,Nova Andradina,2021-06-01,2021-06-01,"{16, 13, 50, 49, 11, 35}",5
1392,2021-07-01,***762.550**,"{36, 04, 12, 45, 57, 44, 10, 28}",AUTO POSTO MAIS COMERCIO DE COMBUSTIVEIS E LUB...,Nova Andradina,2021-07-01,2021-07-01,"{04, 12, 45, 11, 44, 57}",5
3383,2021-02-01,***762.550**,"{03, 01, 23, 02, 07, 48, 37, 54}",AUTO POSTO MAIS COMERCIO DE COMBUSTIVEIS E LUB...,Nova Andradina,2021-02-01,2021-02-01,"{03, 51, 02, 07, 48, 54}",5
4687,2021-08-01,***762.550**,"{23, 38, 19, 40, 43, 47, 35, 54}",AUTO POSTO MAIS COMERCIO DE COMBUSTIVEIS E LUB...,Nova Andradina,2021-08-01,2021-08-01,"{01, 19, 40, 47, 35, 54}",5


In [57]:
df_premiados_cidade = df_premiados.where(df_premiados['mes'] == "2021-08-01")
df_premiados_cidade.dropna()
df_premiados_cidade = df_premiados_cidade.groupby(['acertos','premiados_municipio']).agg({'count'})['premiados_cpf']
df_premiados_cidade = df_premiados_cidade.rename(columns={"count":"ganhadores"})
df_premiados_cidade.sort_values(by=['acertos','premiados_municipio'], ascending=False)

ganhadores
acertos premiados_municipio                 
6.0     Caarapó                            1
5.0     Água Clara                         6
        Três Lagoas                       21
        Tacuru                             1
        São Gabriel do Oeste               6
        Sonora                             2
        Sidrolândia                        5
        Selvíria                           1
        Santa Rita do Pardo                1
        Rio Verde de Mato Grosso           1
        Rio Brilhante                      6
        Ribas do Rio Pardo                 1
        Ponta Porã                         7
        Paranaíba                          3
        Novo Horizonte do Sul              1
        Nova Andradina                    12
        Nova Alvorada do Sul               1
        Nioaque                            2
        Naviraí                            4
        Mundo Novo                         4
        Miranda                            3
        Maracaju                           6
        Jardim                             4
        Ivinhema                           1
        Itaquiraí                          6
        Itaporã                            1
        Inocência                          1
        Iguatemi                           4
        Fátima do Sul                      2
        Dourados                          46
        Deodápolis                         2
        Coxim                              5
        Costa Rica                         4
        Corumbá                            8
        Chapadão do Sul                    4
        Caracol                            1
        Campo Grande                     167
        Camapuã                            6
        Caarapó                            1
        Brasilândia                        1
        Bonito                             1
        Bodoquena                          1
        Bela Vista                         2
        Batayporã                          1
        Bataguassu                         2
        Aquidauana                         3
        Aparecida do Taboado               1
        Angélica                           4
        Anastácio                          4
        Amambai                            5

mes,2020-07-01 00:00:00,2020-08-01 00:00:00,2020-09-01 00:00:00,2020-10-01 00:00:00,2020-11-01 00:00:00,2020-12-01 00:00:00,2021-01-01 00:00:00,2021-02-01 00:00:00,2021-03-01 00:00:00,2021-04-01 00:00:00,2021-05-01 00:00:00,2021-06-01 00:00:00,2021-07-01 00:00:00,2021-08-01 00:00:00,All
premiados_municipio,,,,,,,,,,,,,,,
All,343,365,363,373,394,374,382,345,349,318,350,359,337,383,5035
Campo Grande,142,160,153,170,195,170,170,164,174,132,166,168,149,167,2280
Dourados,39,31,45,46,53,49,42,45,30,35,41,36,31,46,569
Três Lagoas,20,24,19,21,21,16,28,15,17,24,15,21,21,21,283
Corumbá,23,9,14,9,17,16,14,9,13,11,12,18,14,8,187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Figueirão,1,0,0,0,0,0,0,0,0,0,0,1,0,0,2
Aral Moreira,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
Laguna Carapã,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
